This framework is concerned with comparing two sets of data, for instance source brain and target brain.
It does not take care of trying multiple combinations of source data (such as multiple layers in models), but only makes direct comparisons.

## Metrics

A metric tells us how similar to assemblies (sets of data) are to each other.
For comparison, they might be re-mapped (neural predictivity) or compared in sub-spaces (RDMs).

### Pre-defined metrics

Brain-Score comes with many standard metrics used in the field.
For instance, we can easily use regression methods to compare two assemblies based on how well one predicts the neural firing rates in the other:

#### Neural (PLS) Predictivity

In [1]:
import numpy as np

from brainscore.assemblies import NeuroidAssembly
from brainscore.metrics.neural_predictivity import PlsPredictivity

assembly = NeuroidAssembly((np.arange(30 * 25) + np.random.standard_normal(30 * 25)).reshape((30, 25)),
                           coords={'image_id': ('presentation', np.arange(30)),
                                   'object_name': ('presentation', ['a', 'b', 'c'] * 10),
                                   'neuroid_id': ('neuroid', np.arange(25)),
                                   'region': ('neuroid', [0] * 25)},
                           dims=['presentation', 'neuroid'])
metric = PlsPredictivity()
score = metric(source=assembly, target=assembly)
print("\n", score)


cross-validation:   0%|          | 0/10 [00:00<?, ?it/s]

cross-validation:  10%|█         | 1/10 [00:00<00:02,  3.27it/s]

cross-validation:  20%|██        | 2/10 [00:00<00:02,  3.17it/s]

cross-validation:  30%|███       | 3/10 [00:00<00:02,  3.22it/s]

cross-validation:  40%|████      | 4/10 [00:01<00:01,  3.13it/s]

...

cross-validation:  70%|███████   | 7/10 [00:02<00:00,  3.36it/s]

cross-validation:  80%|████████  | 8/10 [00:02<00:00,  3.41it/s]

cross-validation:  90%|█████████ | 9/10 [00:02<00:00,  3.46it/s]

cross-validation: 100%|██████████| 10/10 [00:03<00:00,  3.45it/s]


 Score(_variable=<xarray.Variable (aggregation: 2)>
array([1., 0.])
Attributes:
    raw:      <xarray.DataAssembly (split: 10, neuroid: 25)>\narray([[1., 1.,...,_coords=OrderedDict([('aggregation', <xarray.IndexVariable 'aggregation' (aggregation: 2)>
array(['center', 'error'], dtype='<U6'))]),_name=None,_file_obj=None,_initialized=True)


#### RDM

In [2]:
from brainscore.metrics.rdm import RDMCrossValidated

metric = RDMCrossValidated()
rdm_score = metric(assembly1=assembly, assembly2=assembly)
print("\n", rdm_score)

cross-validation:   0%|          | 0/10 [00:00<?, ?it/s]

cross-validation:  30%|███       | 3/10 [00:00<00:00, 25.53it/s]

cross-validation:  70%|███████   | 7/10 [00:00<00:00, 27.77it/s]

cross-validation: 100%|██████████| 10/10 [00:00<00:00, 31.95it/s]


 Score(_variable=<xarray.Variable (aggregation: 2)>
array([1.000000e+00, 2.482534e-17])
Attributes:
    raw:      <xarray.DataAssembly (split: 10)>\narray([1., 1., 1., 1., 1., 1...,_coords=OrderedDict([('aggregation', <xarray.IndexVariable 'aggregation' (aggregation: 2)>
array(['center', 'error'], dtype='<U6'))]),_name=None,_file_obj=None,_initialized=True)


#### Raw values

Above values are aggregate values over splits and neuroids.
We can also check the raw values.

In [3]:
print(score.attrs['raw'])


<xarray.DataAssembly (split: 10, neuroid: 25)>
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])
Coordinates:
  * split       (split) int64 0 1 2 3 4 5 6 7 8 9
  * neuroid     (neuroid) MultiIndex
  - neuroid_id  (neuroid) int64 0 1 2 3 4 5 6 7 8 ... 16 17 18 19 20 21 22 23 24
  - region      (neuroid) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0


### Custom metrics

A metric simply returns a Score for the similarity of two assemblies.
For instance, the following computes the Euclidean distance of regressed and target neuroids.

In [6]:
from brainscore.assemblies import DataAssembly
from brainscore.metrics.transformations import CrossValidation
from brainscore.metrics.xarray_utils import XarrayRegression
from brainscore.metrics.neural_predictivity import LinearRegression


class DistanceMetric:
    def __init__(self):
        regression = LinearRegression()
        self._regression = XarrayRegression(regression=regression)
        self._cross_validation = CrossValidation()

    def __call__(self, source, target):
        return self._cross_validation(source, target, apply=self._apply, aggregate=self._aggregate)
        
    def _apply(self, source_train, target_train, source_test, target_test):
        self._regression.fit(source_train, target_train)
        prediction = self._regression.predict(source_test)
        score = self._compare(prediction, target_test)
        return score
    
    def _compare(self, prediction, target):
        prediction, target = prediction.sortby('image_id').sortby('neuroid_id'), target.sortby('image_id').sortby('neuroid_id')
        assert all(prediction['image_id'].values == target['image_id'].values)
        assert all(prediction['neuroid_id'].values == target['neuroid_id'].values)
        difference = np.abs(target.values - prediction.values)  # lower is better
        return DataAssembly(difference, coords=target.coords, dims=target.dims)
    
    def _aggregate(self, scores):
        return scores.median('neuroid').mean('presentation')
    

metric = DistanceMetric()
score = metric(assembly, assembly)
print("\n", score)

cross-validation:   0%|          | 0/10 [00:00<?, ?it/s]

cross-validation:  20%|██        | 2/10 [00:00<00:00, 15.30it/s]

cross-validation:  50%|█████     | 5/10 [00:00<00:00, 16.03it/s]

cross-validation:  70%|███████   | 7/10 [00:00<00:00, 16.88it/s]

cross-validation:  90%|█████████ | 9/10 [00:00<00:00, 13.98it/s]

cross-validation: 100%|██████████| 10/10 [00:00<00:00, 14.38it/s]


 Score(_variable=<xarray.Variable (aggregation: 2)>
array([1.017260e-13, 9.840511e-15])
Attributes:
    raw:      <xarray.DataAssembly (split: 10, presentation: 19, neuroid: 25)...,_coords=OrderedDict([('aggregation', <xarray.IndexVariable 'aggregation' (aggregation: 2)>
array(['center', 'error'], dtype='<U6'))]),_name=None,_file_obj=None,_initialized=True)



/home/martin/miniconda3/envs/brainscore/lib/python3.7/site-packages/numpy/lib/function_base.py:3250: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)
